## 1. 导入库

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# 设置随机种子
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

# 设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"PyTorch 版本: {torch.__version__}")
print(f"使用设备: {device}")
print("✅ 库导入完成")

## 2. 加载数据

In [ ]:
# 加载数据
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

# 转换时间戳
train_df['Timestamp'] = pd.to_datetime(train_df['Timestamp'])
test_df['Timestamp'] = pd.to_datetime(test_df['Timestamp'])

# 按时间排序
train_df = train_df.sort_values('Timestamp').reset_index(drop=True)
test_df = test_df.sort_values('Timestamp').reset_index(drop=True)

print(f"训练集形状: {train_df.shape}")
print(f"测试集形状: {test_df.shape}")
print(f"\n训练集时间范围: {train_df['Timestamp'].min()} 到 {train_df['Timestamp'].max()}")
print(f"测试集时间范围: {test_df['Timestamp'].min()} 到 {test_df['Timestamp'].max()}")

## 3. 特征工程

In [ ]:
def create_features(df):
    """创建技术指标特征"""
    df = df.copy()
    
    # 收益率
    df['returns'] = df['Close'].pct_change()
    df['log_returns'] = np.log(df['Close'] / df['Close'].shift(1))
    
    # 移动平均
    for window in [5, 10, 20]:
        df[f'ma_{window}'] = df['Close'].rolling(window=window).mean()
        df[f'ma_ratio_{window}'] = df['Close'] / df[f'ma_{window}']
    
    # 波动率
    df['volatility_10'] = df['returns'].rolling(window=10).std()
    df['volatility_20'] = df['returns'].rolling(window=20).std()
    
    # RSI
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / (loss + 1e-10)
    df['rsi'] = 100 - (100 / (1 + rs))
    
    # MACD
    ema12 = df['Close'].ewm(span=12, adjust=False).mean()
    ema26 = df['Close'].ewm(span=26, adjust=False).mean()
    df['macd'] = ema12 - ema26
    
    # 价格范围
    df['high_low_ratio'] = df['High'] / (df['Low'] + 1e-10)
    df['close_open_ratio'] = df['Close'] / (df['Open'] + 1e-10)
    
    # 成交量
    df['volume_ma_10'] = df['Volume'].rolling(window=10).mean()
    df['volume_ratio'] = df['Volume'] / (df['volume_ma_10'] + 1e-10)
    
    return df

# 创建特征
train_featured = create_features(train_df)
test_featured = create_features(test_df)

print(f"特征后训练集列数: {train_featured.shape[1]}")

In [ ]:
# 定义特征列
feature_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 
                'returns', 'log_returns',
                'ma_ratio_5', 'ma_ratio_10', 'ma_ratio_20',
                'volatility_10', 'volatility_20',
                'rsi', 'macd',
                'high_low_ratio', 'close_open_ratio', 'volume_ratio']

print(f"特征数量: {len(feature_cols)}")
print(f"特征列表: {feature_cols}")

## 4. 数据预处理

In [ ]:
# 删除 NaN
train_clean = train_featured.dropna().reset_index(drop=True)
print(f"清洗后数据: {train_clean.shape}")

# 标准化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(train_clean[feature_cols])
y = train_clean['Target'].values

print(f"X 形状: {X_scaled.shape}")
print(f"y 形状: {y.shape}")

In [ ]:
# 创建序列数据
SEQ_LEN = 60  # 使用过去60个时间步

def create_sequences(X, y, seq_len):
    X_seq, y_seq = [], []
    for i in range(seq_len, len(X)):
        X_seq.append(X[i-seq_len:i])
        y_seq.append(y[i])
    return np.array(X_seq), np.array(y_seq)

X_seq, y_seq = create_sequences(X_scaled, y, SEQ_LEN)
print(f"序列 X 形状: {X_seq.shape}")
print(f"序列 y 形状: {y_seq.shape}")

In [ ]:
# 时序分割
train_size = int(len(X_seq) * 0.8)

X_train = X_seq[:train_size]
y_train = y_seq[:train_size]
X_val = X_seq[train_size:]
y_val = y_seq[train_size:]

print(f"训练集: {X_train.shape}")
print(f"验证集: {X_val.shape}")

# 转为 PyTorch Tensor
X_train_t = torch.FloatTensor(X_train)
y_train_t = torch.FloatTensor(y_train).unsqueeze(1)
X_val_t = torch.FloatTensor(X_val)
y_val_t = torch.FloatTensor(y_val).unsqueeze(1)

# 创建 DataLoader
BATCH_SIZE = 64
train_dataset = TensorDataset(X_train_t, y_train_t)
val_dataset = TensorDataset(X_val_t, y_val_t)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"\n训练 batches: {len(train_loader)}")
print(f"验证 batches: {len(val_loader)}")

## 5. 定义 Transformer 模型

In [ ]:
class CryptoTransformer(nn.Module):
    def __init__(self, feature_size, d_model=64, nhead=4, num_layers=2, dropout=0.1, seq_len=60):
        super(CryptoTransformer, self).__init__()
        
        # 1. Input Embedding: 把特征维度映射到 Transformer 的维度
        self.embedding = nn.Linear(feature_size, d_model)
        
        # 2. Positional Encoding: 让模型知道时间顺序
        # 使用 Learnable Position Embedding
        self.pos_encoder = nn.Parameter(torch.zeros(1, seq_len, d_model))
        
        # 3. Transformer Encoder
        encoder_layers = nn.TransformerEncoderLayer(
            d_model, nhead, 
            dim_feedforward=d_model*4, 
            dropout=dropout, 
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        
        # 4. Output Head
        self.decoder = nn.Sequential(
            nn.Linear(d_model * seq_len, 128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128, 1)
        )
        
        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.decoder[0].weight.data.uniform_(-initrange, initrange)
        self.decoder[3].weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        # src shape: [Batch, Seq_Len, Features]
        
        # Embedding
        x = self.embedding(src)  # -> [Batch, Seq_Len, d_model]
        
        # Add Position info
        x = x + self.pos_encoder
        
        # Transformer Processing
        x = self.transformer_encoder(x)  # -> [Batch, Seq_Len, d_model]
        
        # Flatten
        x = x.reshape(x.size(0), -1)  # -> [Batch, Seq_Len * d_model]
        
        # Prediction
        output = self.decoder(x)
        return output

# 创建模型
model = CryptoTransformer(
    feature_size=len(feature_cols),
    d_model=64,
    nhead=4,
    num_layers=2,
    dropout=0.1,
    seq_len=SEQ_LEN
).to(device)

print(model)
print(f"\n模型参数量: {sum(p.numel() for p in model.parameters()):,}")

## 6. 训练模型

In [ ]:
# 损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

# 训练参数
EPOCHS = 20
best_val_loss = float('inf')
patience = 5
patience_counter = 0

train_losses = []
val_losses = []

print("开始训练...\n")

In [ ]:
for epoch in range(EPOCHS):
    # 训练阶段
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        train_loss += loss.item()
    
    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    
    # 验证阶段
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            output = model(X_batch)
            loss = criterion(output, y_batch)
            val_loss += loss.item()
    
    val_loss /= len(val_loader)
    val_losses.append(val_loss)
    
    # 学习率调整
    scheduler.step(val_loss)
    
    print(f"Epoch {epoch+1}/{EPOCHS} - Train Loss: {train_loss:.6f} - Val Loss: {val_loss:.6f}")
    
    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), '../models/transformer_best.pth')
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"\nEarly stopping at epoch {epoch+1}")
            break

print("\n✅ 训练完成!")

In [ ]:
# 绘制训练曲线
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training History')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 7. 模型评估

In [ ]:
# 加载最佳模型
model.load_state_dict(torch.load('../models/transformer_best.pth'))
model.eval()

# 验证集预测
with torch.no_grad():
    val_pred = model(X_val_t.to(device)).cpu().numpy().flatten()

# 评估
rmse = np.sqrt(mean_squared_error(y_val, val_pred))
r2 = r2_score(y_val, val_pred)
corr = np.corrcoef(y_val, val_pred)[0, 1]

print(f"验证集评估:")
print(f"  RMSE: {rmse:.6f}")
print(f"  R²: {r2:.6f}")
print(f"  Correlation: {corr:.6f}")

## 8. 测试集预测

In [ ]:
# 准备测试数据
# 需要训练集尾部数据来构建序列
train_tail = train_clean.tail(SEQ_LEN + 100).copy()

# 合并
test_with_history = pd.concat([train_tail, test_featured], ignore_index=True)
test_with_history = test_with_history.fillna(method='ffill').fillna(method='bfill')

# 标准化
X_test_scaled = scaler.transform(test_with_history[feature_cols])

# 创建序列
X_test_seq = []
for i in range(SEQ_LEN, len(X_test_scaled)):
    X_test_seq.append(X_test_scaled[i-SEQ_LEN:i])
X_test_seq = np.array(X_test_seq)

# 只取测试集部分
n_test = len(test_df)
X_test_seq = X_test_seq[-n_test:]

print(f"测试序列形状: {X_test_seq.shape}")

In [ ]:
# 预测
X_test_t = torch.FloatTensor(X_test_seq).to(device)

model.eval()
with torch.no_grad():
    test_predictions = model(X_test_t).cpu().numpy().flatten()

print(f"预测数量: {len(test_predictions)}")
print(f"预测均值: {test_predictions.mean():.6f}")
print(f"预测标准差: {test_predictions.std():.6f}")

## 9. 生成提交文件

In [ ]:
# 创建提交文件
submission_df = pd.DataFrame({
    'Timestamp': test_df['Timestamp'].values[:len(test_predictions)],
    'Prediction': test_predictions
})

# 保存
submission_dir = Path('../submissions')
submission_dir.mkdir(exist_ok=True)

submission_file = submission_dir / 'transformer_submission.csv'
submission_df.to_csv(submission_file, index=False)

print(f"✅ 提交文件已保存: {submission_file}")
print(f"\n提交文件预览:")
print(submission_df.head(10))
print(f"\n预测统计:")
print(submission_df['Prediction'].describe())

## 10. 保存模型

In [ ]:
# 保存最终模型
model_dir = Path('../models')
model_dir.mkdir(exist_ok=True)

torch.save({
    'model_state_dict': model.state_dict(),
    'feature_cols': feature_cols,
    'seq_len': SEQ_LEN,
}, model_dir / 'transformer_model.pth')

import joblib
joblib.dump(scaler, model_dir / 'transformer_scaler.pkl')

print("✅ 模型已保存")

## 总结

### Transformer 模型架构:
1. **Input Embedding**: 将特征映射到 d_model 维度
2. **Learnable Position Encoding**: 学习时间位置信息
3. **Transformer Encoder**: 自注意力机制捕捉时序依赖
4. **Decoder Head**: 全连接层输出预测值

### 优势:
- 自注意力机制能捕捉长距离依赖
- 并行计算效率高
- 可解释性好（注意力权重）